In [1]:
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential
from keras.layers import Dropout, Flatten, Dense
from keras.models import Model
import os
import keras
import math
import numpy as np
from keras.utils.np_utils import to_categorical

use_full = False

# path to the model weights files.
weights_path = '../keras/examples/vgg16_weights.h5'
#top_model_weights_path = '../model/bottleneck_fc_model.h5'
top_model_weights_path = '../model/bottleneck_fc_model_inceptionv3_500_val_acc_4791.h5'
#top_model_weights_path = '../model/fc_model.h5'
img_width, img_height = 500, 500

epochs = 1
batch_size = 64

image_train_file = '../data/image_train.npy'
image_validation_file = '../data/image_validation.npy'
image_test_file = '../data/image_test.npy'

#full
if use_full:
    train_data_dir = '../data/Pandora18K_train_val_test_split/train'
    validation_data_dir = '../data/Pandora18K_train_val_test_split/val'
    test_data_dir = '../data/Pandora18K_train_val_test_split/test'
    nb_train_samples = 14313
    nb_validation_samples = 1772
    nb_test_samples = 1791
    total_num_classes = 18
    validation_labels = np.array([0]*72+[1]*73+[2]*72+[3]*93+[4]*78+[5]*74+[6]*85+[7]*124+[8]*131+[9]*118+[10]*109+[11]*105+[12]*80+[13]*130+[14]*108+[15]*89+[16]*111+[17]*120)
    train_labels = np.array([0]*684+[1]*598+[2]*655+[3]*657+[4]*808+[5]*675+[6]*715+[7]*946+[8]*995+[9]*1021+[10]*803+[11]*816+[12]*566+[13]*959+[14]*842+[15]*831+[16]*849+[17]*893)
    test_labels = np.array([0]*91+[1]*60+[2]*75+[3]*82+[4]*104+[5]*83+[6]*95+[7]*121+[8]*131+[9]*123+[10]*103+[11]*117+[12]*65+[13]*123+[14]*121+[15]*112+[16]*89+[17]*96)
else:
    train_data_dir = '../data/Pandora18K_small_train_val_test_split/train'
    validation_data_dir = '../data/Pandora18K_small_train_val_test_split/val'
    test_data_dir = '../data/Pandora18K_small_train_val_test_split/test'
    nb_train_samples = 1462
    nb_validation_samples = 167
    nb_test_samples = 171
    total_num_classes = 18
    validation_labels = np.array([0]*8+[1]*10+[2]*11+[3]*5+[4]*11+[5]*6+[6]*8+[7]*8+[8]*9+[9]*12+[10]*7+[11]*10+[12]*14+[13]*5+[14]*11+[15]*11+[16]*12+[17]*9)
    train_labels = np.array([0]*78+[1]*79+[2]*77+[3]*85+[4]*78+[5]*87+[6]*80+[7]*81+[8]*81+[9]*80+[10]*85+[11]*83+[12]*74+[13]*87+[14]*83+[15]*83+[16]*78+[17]*83)
    test_labels = np.array([0]*14+[1]*11+[2]*12+[3]*10+[4]*11+[5]*7+[6]*12+[7]*11+[8]*10+[9]*8+[10]*8+[11]*7+[12]*12+[13]*8+[14]*6+[15]*6+[16]*10+[17]*8)

validation_labels = to_categorical(validation_labels, num_classes=18)
train_labels = to_categorical(train_labels, num_classes=18)
test_labels = to_categorical(test_labels, num_classes=18)
    


Using TensorFlow backend.


In [2]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale= 1./255)

valid_datagen = ImageDataGenerator(rescale=1. / 255)
test_datagen  = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle = False)

validation_generator = valid_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle = False)

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle = False)

Found 1462 images belonging to 18 classes.
Found 167 images belonging to 18 classes.
Found 171 images belonging to 18 classes.


In [3]:
#Get the train image inputs as numpy arrays

datagen = ImageDataGenerator(rescale=1. / 255)
model = Sequential()
model.add(Dropout(0,input_shape = (img_width,img_height,3)))

generator = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)
img_train = model.predict_generator(
    generator, (nb_train_samples // batch_size)+1, verbose=True, pickle_safe=True, workers=1)
print(img_train.shape)
del model
np.save(open(image_train_file, 'w'), img_train)


Found 1462 images belonging to 18 classes.
23/23 [==============================] - 16s    
(1462, 500, 500, 3)


In [4]:
#Get the validation image inputs as numpy arrays

datagen = ImageDataGenerator(rescale=1. / 255)
model = Sequential()
model.add(Dropout(0,input_shape = (img_width,img_height,3)))

generator = datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)
img_validation = model.predict_generator(
    generator, (nb_validation_samples // batch_size)+1, verbose=True, pickle_safe=True, workers=1)
print(img_validation.shape)
del model
np.save(open(image_validation_file, 'w'), img_validation)

Found 167 images belonging to 18 classes.
3/3 [==============================] - 2s     
(167, 500, 500, 3)


In [5]:
#Get the test image inputs as numpy arrays

datagen = ImageDataGenerator(rescale=1. / 255)
model = Sequential()
model.add(Dropout(0,input_shape = (img_width,img_height,3)))

generator = datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode=None,
    shuffle=False)
img_test = model.predict_generator(
    generator, (nb_test_samples // batch_size)+1, verbose=True, pickle_safe=True, workers=1)
print(img_test.shape)
del model
np.save(open(image_test_file, 'w'), img_test)

Found 171 images belonging to 18 classes.
3/3 [==============================] - 2s     
(171, 500, 500, 3)


In [6]:
train_data = np.load(open(image_train_file))
print("\ntrain_data.shape")
print(train_data.shape)
print("train_labels.shape")
print(train_labels.shape)

validation_data = np.load(open(image_validation_file))
print("\nvalidation_data.shape")
print(validation_data.shape)
print("validation_labels.shape")
print(validation_labels.shape)

test_data = np.load(open(image_test_file))
print("\ntest_data.shape")
print(test_data.shape)
print("test_labels.shape")
print(test_labels.shape)


train_data.shape
(1462, 500, 500, 3)
train_labels.shape
(1462, 18)

validation_data.shape
(167, 500, 500, 3)
validation_labels.shape
(167, 18)

test_data.shape
(171, 500, 500, 3)
test_labels.shape
(171, 18)


In [7]:
pretrained_model = "part_3_weights"
weight_dir = "../model/%s"%pretrained_model
if not os.path.exists(weight_dir):
    os.makedirs(weight_dir)

best_weight_path = "../model/best.h5"
best_acc = 0
best_lr = 0
best_momentum = 0
best_frozen = 0
num_parameter_sets = 1
best_model = None
    
for i in range(num_parameter_sets):
    print("\n")
    
    lr = 10**-8#10**np.random.uniform(low=-7, high=-5)
    momentum = 0.9#np.random.uniform(low = 0.8, high = 0.9)
    num_frozen = 312#np.random.randint(300, high=312) #312 total
    print("lr = " + str(lr))
    print("momentum = " + str(momentum))
    print("number of frozen layers = " + str(num_frozen))
    
    # build the VGG16 network
    base_model = applications.inception_v3.InceptionV3(include_top=False, weights='imagenet', input_shape=(img_height,img_width,3))

    # build a classifier model to put on top of the convolutional model
    top_model = Sequential()
    top_model.add(Flatten(input_shape=base_model.output_shape[1:]))
    top_model.add(Dropout(0.6))
    top_model.add(Dense(256, activation='relu'))
    top_model.add(Dropout(.6))
    top_model.add(Dense(18, activation='softmax'))

    top_model.load_weights(top_model_weights_path)

    model = Model(inputs= base_model.input, outputs= top_model(base_model.output))
    
    for layer in model.layers[:num_frozen]:
        layer.trainable = False
    for layer in model.layers[280:]:
        layer.trainable = True
    for layer in model.layers[311].layers:
        layer.trainable = True

    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizers.SGD(lr=lr, momentum=momentum),
                  metrics=['accuracy'])
    
    param_names = "lr_" + str(lr)
    weight_path = os.path.join(weight_dir, param_names + "_best_weights%s.h5"%pretrained_model)
    checkpointer = keras.callbacks.ModelCheckpoint(filepath=weight_path, verbose=1,monitor='val_acc', save_best_only=True, save_weights_only=True)

    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels),
              callbacks=[checkpointer])
    
    """
    model.fit_generator(
        train_generator,
        steps_per_epoch=(nb_train_samples // batch_size)+1,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=(nb_validation_samples // batch_size)+1,
        callbacks=[checkpointer, stopper])
    """
    
    model.load_weights(weight_path)
    new_loss, new_acc = model.evaluate(validation_data, validation_labels, batch_size=batch_size, verbose=1, sample_weight=None)
    #new_loss, new_acc = model.evaluate_generator(generator = validation_generator, steps = (nb_validation_samples // batch_size)+1)
    print("new_acc = "+ str(new_acc))
    print("new_loss = "+ str(new_loss))
    
    if new_acc > best_acc:
        model.load_weights(weight_path)
        model.save_weights(best_weight_path, overwrite = True)
        best_acc = new_acc
        best_lr = lr
        best_momentum = momentum
        best_frozen = num_frozen
        best_model = model
    #else:
        #del model
        
    os.remove(weight_path)

    
    print("best val acc so far = " + str(best_acc))
    



lr = 1e-08
momentum = 0.9
number of frozen layers = 312
Train on 1462 samples, validate on 167 samples
Epoch 1/1
167/167 [==============================] - 7s     
new_acc = 0.808383236388
new_loss = 0.814803152741
best val acc so far = 0.808383236388


In [8]:
best_model.load_weights(best_weight_path)

best_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=best_lr, momentum=best_momentum),
              metrics=['accuracy'])

val_loss, val_acc = best_model.evaluate(validation_data, validation_labels, batch_size=batch_size, verbose=1, sample_weight=None)
print("best val loss = " + str(val_loss))
print("best val acc = " + str(val_acc))

167/167 [==============================] - 8s     
best val loss = 0.814803152741
best val acc = 0.808383236388


In [9]:
best_model.load_weights(best_weight_path)

best_model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=best_lr, momentum=best_momentum),
              metrics=['accuracy'])

test_loss, test_acc = best_model.evaluate(test_data, test_labels, batch_size=batch_size, verbose=1, sample_weight=None)
print("best test loss = " + str(test_loss))
print("best test acc = " + str(test_acc))

171/171 [==============================] - 11s    
best test loss = 0.853165199882
best test acc = 0.730994150304


In [ ]:
#THIS IS NOT READY YET

from keras.utils.np_utils import to_categorical

#y_test = to_categorical(validation_labels, num_classes=18)
y_test = validation_labels
prob_predict = model.predict_generator(generator = validation_generator, steps = (nb_validation_samples / batch_size) + 1, max_q_size=10, workers=1, pickle_safe=False, verbose=0)
y_pred = np.argmax(prob_predict, axis=1)

print(y_test.shape)
print(prob_predict.shape)
print(y_pred.shape)
print(y_pred)
print(y_test)

"""
from sklearn.metrics import confusion_matrix
y_true = [2, 0, 2, 2, 0, 1]
y_pred = [0, 0, 2, 2, 0, 2]

confusion_matrix(y_true, y_pred)

y_true = ["cat", "ant", "cat", "cat", "ant", "bird"]
y_pred = ["ant", "ant", "cat", "cat", "ant", "cat"]
confusion_matrix(y_true, y_pred, labels=["ant", "bird", "cat"])
"""

In [ ]:
print(__doc__)

import itertools
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# import some data to play with
"""
iris = datasets.load_iris()
X = iris.data
y = iris.target
class_names = iris.target_names

# Split the data into a training set and a test set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# Run classifier, using a model that is too regularized (C too low) to see
# the impact on the results
classifier = svm.SVC(kernel='linear', C=0.01)
y_pred = classifier.fit(X_train, y_train).predict(X_test)
"""

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

class_names = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
print(class_names)
print(y_test.shape)
print(y_pred.shape)
    
# Compute confusion matrix
cnf_matrix = confusion_matrix(y_test, y_pred)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()